In [ ]:
#pip install supabase

In [ ]:
#pip install pandas

In [ ]:
#pip install python-dotenv

1. Supabase 셋업

In [1]:
import os
import dotenv
import pandas as pd
from supabase import create_client, Client
import json
import ast

dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

url: str = os.environ["SUPABASE_URL"]
key: str = os.environ["SUPABASE_KEY"]
supabase: Client = create_client(url, key)

2. 파일명과 테이블명 입력 및 데이터프레임 셋업

In [2]:
data_file = "Exercise.csv"
table_name = "Exercise_duplicate"

update_data = pd.read_csv(data_file)
response = supabase.table(table_name).select("*").execute()
origin_data = pd.DataFrame(response.data)

# 데이터 확인
print(update_data.shape)
print(origin_data.shape)
update_data.head()
origin_data.head()

(208, 22)
(149, 22)


,id,Name,Description,Level,BeginnerTime,BeginnerCount,TargetMuscle,AnimationFile,ClipSource,ImageFile,...,Type,BasicCount,IntermediateCount,AdvancedCount,UpperCalibration,TargetDevice,LowerCalibration,exCategory,Multiply,Side
0,5,Push-Up,A classic push-up exercise for chest.,보통,30,15,"[Chest, Triceps]",https://nherivcnanphkalgybji.supabase.co/stora...,https://youtu.be/fBh97ozxNGw,https://nherivcnanphkalgybji.supabase.co/stora...,...,둘다,15,15,15,NaN,None,NaN,1,NaN,NaN
1,4,코브라 스트레칭,엎드려 누운 상태에서 두 손을 어깨 밑에 놓고 팔꿈치를 굽힙니다. 그리고 가슴을 바...,입문,30,1,"[복직근, 상완삼두근, 완요골근]",https://nherivcnanphkalgybji.supabase.co/stora...,https://youtu.be/YyjQ_-e7nu8,https://nherivcnanphkalgybji.supabase.co/stora...,...,둘다,15,15,15,NaN,"[1, 2]",NaN,1,NaN,NaN
2,6,Squat,A basic squat exercise.,쉬움,30,20,"[Quads, Hamstrings]",https://nherivcnanphkalgybji.supabase.co/stora...,https://youtu.be/fBh97ozxNGw,https://nherivcnanphkalgybji.supabase.co/stora...,...,하체,15,15,15,NaN,None,NaN,1,NaN,NaN
3,7,Plank,Hold a plank position to strengthen the core.,어려움,30,1,"[복직근, 복사근, 비복근]",https://nherivcnanphkalgybji.supabase.co/stora...,https://youtu.be/fBh97ozxNGw,https://nherivcnanphkalgybji.supabase.co/stora...,...,둘다,15,15,15,NaN,None,NaN,1,NaN,NaN
4,10,Crunch,A classic abdominal exercise.,보통,30,15,[복직근],https://nherivcnanphkalgybji.supabase.co/stora...,https://youtu.be/fBh97ozxNGw,https://nherivcnanphkalgybji.supabase.co/stora...,...,상체,15,15,15,NaN,None,NaN,1,NaN,NaN


3. 기존에 없는 새로 추가된 record 추출

In [3]:
# DB에 있는 데이터 중에서 없는 새로운 데이터 찾기
new_records =  update_data[~update_data['id'].isin(origin_data['id'])]
insert_data = new_records.drop(columns=['id'])

# Name, Description, exCategory를 제외한 NaN처리 컬럼 선택
non_fill = insert_data.columns.difference(['Name', 'Description', 'exCategory'])
insert_data[non_fill] = insert_data[non_fill].fillna(0)

# 데이터 확인
print(new_records.shape)
print(insert_data.shape)
print(insert_data)

(86, 22)
(86, 21)
               Name                                        Description Level  \
122       스쿼트 사이드 잽                                                테스트    보통   
123        니 허그 크런치  누워서 한 쪽 무릎을 가슴 쪽으로 최대한 당기면서 상체를 살짝 앞으로 숙여 줍니다....    보통   
124       풀다운 니업(L)                                                테스트    보통   
125      사이드 터치 스쿼트  사이드 스텝으로 옆으로 이동하면서 매 스텝마다 스쿼트 자세를 취하고 두 손가락으로 ...    보통   
126        스모 턴 스쿼트  스모 스쿼트 자세에서 시작하여, 스쿼트를 하면서 180도 회전하고 다시 스쿼트를 합...    보통   
..              ...                                                ...   ...   
203       풀다운 니업(R)                                                테스트    보통   
204         딥 런지(R)                                                테스트    쉬움   
205    다운 도그 크런치(R)                                                테스트    힘듬   
206  사이드 플랭크 크런치(R)                                                테스트    보통   
207  스탭 백 니 쓰러스트(R)                                                테스트    힘듬   

     BeginnerTime  Be

4. 입력 데이터 맞춤

In [4]:
# Side, UpperCalibration, LowerCalibration 컬럼의 데이터 타입을 int로 변환
insert_data['Side'] = insert_data['Side'].astype(int)
insert_data['UpperCalibration'] = insert_data['UpperCalibration'].astype(int)
insert_data['LowerCalibration'] = insert_data['LowerCalibration'].astype(int)

# Supabase insert를 위해 데이터프레임을 딕셔너리 리스트로 변환
insert_dict = insert_data.to_dict(orient='records')

# 배열 형식으로 변환
# for record in insert_dict:
#  record['TargetMuscle'] = record['TargetMuscle']
#  record['Part'] = record['Part']
#  record['TargetDevice'] = record['TargetDevice']
 
 # 배열 형식으로 변환
for record in insert_dict:
 record['TargetMuscle'] = json.loads(json.dumps(record['TargetMuscle']))
 record['Part'] = json.loads(json.dumps(record['Part']))
 record['TargetDevice'] = json.loads(json.dumps(record['TargetDevice']))
 
print(insert_dict[0]['Part'])
print(insert_dict[0]['UpperCalibration'])
print(insert_dict[0]['TargetMuscle'])

["허벅지"]
0
["대퇴사두근","대흉근","삼두근"]


In [5]:
insert_dict[0]

{'Name': '스쿼트 사이드 잽',
 'Description': '테스트',
 'Level': '보통',
 'BeginnerTime': 30,
 'BeginnerCount': 10,
 'TargetMuscle': '["대퇴사두근","대흉근","삼두근"]',
 'AnimationFile': 0.0,
 'ClipSource': 'https://youtu.be/7YLCPyUXwX8',
 'ImageFile': 0.0,
 'MET': 4.0,
 'Part': '["허벅지"]',
 'Type': '하체',
 'BasicCount': 10,
 'IntermediateCount': 30,
 'AdvancedCount': 40,
 'UpperCalibration': 0,
 'TargetDevice': '[3,4]',
 'LowerCalibration': 1,
 'exCategory': 1,
 'Multiply': 1,
 'Side': 0}

In [6]:
array_columns = ['TargetMuscle', 'Part', 'TargetDevice']
# 문자열을 실제 배열로 변환
def parse_array(col):
    return col.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

for col in array_columns:
    if col in insert_data.columns:
        insert_data[col] = parse_array(insert_data[col])

In [7]:
insert_dict[0]

{'Name': '스쿼트 사이드 잽',
 'Description': '테스트',
 'Level': '보통',
 'BeginnerTime': 30,
 'BeginnerCount': 10,
 'TargetMuscle': '["대퇴사두근","대흉근","삼두근"]',
 'AnimationFile': 0.0,
 'ClipSource': 'https://youtu.be/7YLCPyUXwX8',
 'ImageFile': 0.0,
 'MET': 4.0,
 'Part': '["허벅지"]',
 'Type': '하체',
 'BasicCount': 10,
 'IntermediateCount': 30,
 'AdvancedCount': 40,
 'UpperCalibration': 0,
 'TargetDevice': '[3,4]',
 'LowerCalibration': 1,
 'exCategory': 1,
 'Multiply': 1,
 'Side': 0}

In [8]:
data, count = supabase.table(table_name).upsert(insert_dict).execute()
print(count)

APIError: {'code': '22P02', 'details': '"[" must introduce explicitly-specified array dimensions.', 'hint': None, 'message': 'malformed array literal: "["대퇴사두근","대흉근","삼두근"]"'}

In [32]:
for record in insert_dict:
    insert_response = supabase.table(table_name).insert(record).execute()
    print("Insert reponse:", insert_response)


APIError: {'code': '22P02', 'details': '"[" must introduce explicitly-specified array dimensions.', 'hint': None, 'message': 'malformed array literal: "["대퇴사두근","대흉근","삼두근"]"'}